In [1]:
!pip install yfinance --upgrade --no-cache-dir
!pip install ta

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 63 kB 6.2 MB/s 
     |████████████████████████████████| 6.4 MB 15.7 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel 

In [1]:
import yfinance as yf
import numpy as np
import ta
import pandas as pd
import matplotlib.pyplot as plt 
import plotly.graph_objs as plty
from datetime import datetime

In [2]:
def createFrame(name, start):
  df = yf.download(name, start, interval='1h')
  #fast stochastic --> 
  day = np.arange(1, len(df) + 1)
  df['Day'] = day
  df['%K'] = ta.momentum.stoch(df.High, df.Low, df.Close, window=14, smooth_window=3)
  #slow stochastic --> 3 period simple moving average 
  #mean of 3 time steps 
  df['%D'] = df['%K'].rolling(3).mean()
  df['RSI'] = ta.momentum.rsi(df.Close, window=14)
  #Difference between the MACD and signal line 
  df['MACD'] = ta.trend.macd_diff(df.Close)
  df['Middle Band'] = df['Close'].rolling(window=20).mean()
  df['Upper Band'] = df['Middle Band'] + 1.96 * df['Close'].rolling(window=21).std()
  df['Lower Band'] = df['Middle Band'] - 1.96 * df['Close'].rolling(window=21).std()
  df['12-day MA'] = df['Close'].rolling(12).mean()
  df['26-day MA'] = df['Close'].rolling(26).mean()

  df.dropna(inplace=True)
  return df

In [3]:
def buyDates(df):

    #lags --> number of timesteps that you want to go back
    #Make global     
    lags  = 14
   
    #Crossings of the K and D lines 
    #Temp data frame
    dfx = pd.DataFrame()
    for i in range(1,lags+1):
        #make 20 global
        mask = (df['%K'].shift(i) < 20) & (df['%D'].shift(i) < 20)   
        dfx = dfx.append(mask, ignore_index=True) 
        
    #if the vertical sum of this dataframe is above 0 then there was a cross which indicates a buying trigger 

    #If buy then 1 else value is 0 
    df['stochBuy'] = np.where(dfx.sum(axis=0), 1, 0)
    #Explain why stochastic is between 20 and 80
    df['Buy'] = np.where((df.stochBuy) & (df['%K'].between(20,80)) & (df['%D'].between(20,80)) & (df['RSI'] >= 50) & (df['RSI'] <= 70) 
                        & (df.MACD > 0) & (df['12-day MA'] > df['26-day MA']), 1, 0)

    #Loads the buy signals with the necessary dates
    buyingDates = []
    for i in range(len(df) - 1):
    #Buy column in each row contains buy signal
        if df.Buy.iloc[i]:
            #forwarding looking bias without the +1 
            # +1 allows to buy in the time stamp in which indicator is being evaluated
            buyingDates.append(df.iloc[i + 1].name)

    
    return df, buyingDates

In [4]:
def plotter(df,buyingDates, startDate):
 
  fig = plty.Figure()
  fig.add_trace(plty.Scatter(mode='markers',x=buyingDates, y=df.Open[buyingDates], 
                             marker=dict(color='light green',
            size=14, line=dict(color='green',width=1)), showlegend=True, name='Buy Points')) 
  

  fig.add_trace(plty.Scatter(x=df.index, y=df['Open'], line=dict(color='lightblue', smoothing=1.3, width=1.5), name='Price'))

  fig.add_trace(plty.Candlestick(x = df.index, open=df['Open'],high=df['High'], low=df['Low'],
                              close=df['Close'], name='Candle Sticks'))

  fig.update_layout(
    width=1000,
    height=800,
    title = name + " Chart",
    xaxis_title ='Time (Weeks since ' + startDate + ")",
    yaxis_title = name + ' Share Price (USD)'
  )

  fig.show()

In [5]:
name =  input("Provide a stock name: "  )
date = input("Provide a start date (with this format YYYY-MM-DD): ")

df = createFrame(name, date)
df, buyingDates = buyDates(df)

if  df.Open[buyingDates].empty:
  print("No buy dates for " + name + " between " + date + " to now.")
  plotter(df, buyingDates, date)
else:
    plotter(df, buyingDates, date)
    rv = df.Open[buyingDates]
    print("Buying Dates: Price")
    print(rv)

[*********************100%***********************]  1 of 1 completed


Buying Dates: Price
2022-03-03 11:30:00-05:00    166.649994
2022-03-03 12:30:00-05:00    166.250000
2022-03-03 15:30:00-05:00    166.679993
2022-04-20 13:30:00-04:00    167.410004
2022-04-20 14:30:00-04:00    167.139999
2022-04-20 15:30:00-04:00    167.434998
Name: Open, dtype: float64
